In [2]:
import pandas as pd
from pathlib import Path

In [3]:
# -------------------------------------------------------------------
# 1. Load the data
# -------------------------------------------------------------------
file_path = Path("/mnt/data-r1/data/ITUR/ITUR_resultados_nacional_v1.csv")
df = pd.read_csv(file_path)

In [4]:
n_rows, n_cols = df.shape
print(f"\nRows: {n_rows:,d}")
print(f"Columns ({n_cols}): {list(df.columns)}")


Rows: 9,006,161
Columns (10): ['COD_CELDA', 'TAM_POB', 'DEN_POB', 'DILOCCON50', 'CAR_SER_VI', 'P_USOSUEPV', 'USO_SUECON', 'COND_ACCE', 'EQUIP_URB', 'RESUL_ITUR']


In [5]:
df.head()

,COD_CELDA,TAM_POB,DEN_POB,DILOCCON50,CAR_SER_VI,P_USOSUEPV,USO_SUECON,COND_ACCE,EQUIP_URB,RESUL_ITUR
0,O248023772,,0.0,118127.032715,1.0,0.216889,0.0,0.0,0.0,0.986936
1,O248205762,,0.0,80679.900000,1.0,0.216889,0.0,0.0,0.0,0.983921
2,O256124660,,0.0,111098.099219,1.0,0.216889,0.0,0.0,0.0,0.985931
3,O238203863,,0.0,83422.312812,1.0,0.216888,0.0,0.0,0.0,0.983921
4,O247113763,,0.0,77315.489720,1.0,0.216888,0.0,0.0,0.0,0.982916


In [6]:
path_itur = Path("/mnt/data-r1/data/ITUR/ITUR_resultados_nacional_v1.csv")
path_xy   = Path("/mnt/data-r1/JoaquinSalas/Documents/informs/research/2025.06.05ITUR/data/INEGI_CPV2020_n9_XY.csv")
path_out  = Path("/mnt/data-r1/data/ITUR_dataset.csv")   # <-- cambia si lo necesitas

In [7]:
# -----------------------------------------------------------
# 2. Cargar los datos
# -----------------------------------------------------------
df_itur = pd.read_csv(path_itur, dtype={"COD_CELDA": str})
df_xy   = pd.read_csv(path_xy,   dtype={"CODIGO":    str})

In [20]:
df_itur.shape

(9006161, 10)

In [14]:
df_xy.shape

(697575, 3)

In [9]:
# -----------------------------------------------------------
# 3. Renombrar y fusionar para buscar correspondencias
# -----------------------------------------------------------
df_xy = df_xy.rename(columns={"CODIGO": "COD_CELDA"})   # ahora ambas tablas comparten la clave
merged = pd.merge(df_xy, df_itur[["COD_CELDA", "RESUL_ITUR"]],
                  on="COD_CELDA", how="left", indicator=True)

In [10]:
# -----------------------------------------------------------
# 4. Estadísticas de correspondencia
# -----------------------------------------------------------
total_xy   = len(df_xy)
matches    = (merged["_merge"] == "both").sum()
missed_xy  = total_xy - matches           # registros XY sin pareja en ITUR
missed_msg = f"""
Total XY         : {total_xy:,}
Coincidencias    : {matches:,}
Sin coincidencia : {missed_xy:,}
"""
print(missed_msg.strip())

Total XY         : 697,575
Coincidencias    : 697,575
Sin coincidencia : 0


In [12]:
# -----------------------------------------------------------
# 5. Filtrar los que sí coinciden y guardar dataset final
# -----------------------------------------------------------
final_df = merged.loc[merged["_merge"] == "both",
                      ["COD_CELDA", "X", "Y", "RESUL_ITUR"]]

final_df.to_csv(path_out, index=False)
print(f"\nDataset guardado en: {path_out}")
print(f"Filas escritas: {len(final_df):,}")


Dataset guardado en: /mnt/data-r1/data/ITUR_dataset.csv
Filas escritas: 697,575


In [19]:
# suponiendo que ya tienes df_itur y df_xy cargados
unmatched_mask   = ~df_itur["COD_CELDA"].isin(df_xy["COD_CELDA"])
unmatched_count  = unmatched_mask.sum()

print(f"Registros en df_itur sin correspondencia en df_xy: {unmatched_count:,}")


Registros en df_itur sin correspondencia en df_xy: 8,308,586


In [18]:
df_xy.columns

Index(['COD_CELDA', 'X', 'Y'], dtype='object')

In [21]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------
# 1.  Cargar archivos (ajusta las rutas si difieren)
# ------------------------------------------------------------------
df_itur = pd.read_csv("/mnt/data-r1/data/ITUR/ITUR_resultados_nacional_v1.csv",
                      dtype={"COD_CELDA": str})
df_xy   = pd.read_csv("/mnt/data-r1/JoaquinSalas/Documents/informs/research/2025.06.05ITUR/data/INEGI_CPV2020_n9_XY.csv",
                      dtype={"CODIGO": str})

print("Filas en df_itur :", len(df_itur))
print("Filas en df_xy   :", len(df_xy))

# ------------------------------------------------------------------
# 2.  Conteo “fila a fila” (no se duplica nada)
# ------------------------------------------------------------------
mask_match      = df_itur["COD_CELDA"].isin(df_xy["CODIGO"])
n_match_rows    = mask_match.sum()
n_unmatch_rows  = (~mask_match).sum()

print("\n=== FILA a FILA (correcto) ===")
print("Coincidencias (filas de df_itur):", f"{n_match_rows:,}")
print("Sin coincidencia                :", f"{n_unmatch_rows:,}")
print("Suma                            :", f"{n_match_rows + n_unmatch_rows:,}")

# ------------------------------------------------------------------
# 3.  Conteo vía merge()  ➜  los duplicados de CODIGO inflan el total
# ------------------------------------------------------------------
merged = pd.merge(df_itur,            # ¡df_itur se puede replicar!
                  df_xy.rename(columns={"CODIGO": "COD_CELDA"}),
                  on="COD_CELDA",
                  how="inner")

print("\n=== CON merge() (duplica filas) ===")
print("Filas en el inner-merge:", f"{len(merged):,}")

# ------------------------------------------------------------------
# 4.  ¿Cuántos CODIGO están duplicados en df_xy?
# ------------------------------------------------------------------
dup_xy = df_xy["CODIGO"].duplicated(keep=False)   # marca TODAS las repeticiones
n_dup_codes = dup_xy.sum()                        # número de filas duplicadas
n_uniq_dup  = df_xy.loc[dup_xy, "CODIGO"].nunique()  # cuántos códigos distintos
print("\nDuplicados en df_xy:")
print("· Filas que se repiten  :", f"{n_dup_codes:,}")
print("· Códigos distintos rep. :", f"{n_uniq_dup:,}")

# ------------------------------------------------------------------
# 5.  Verificación de la discrepancia
# ------------------------------------------------------------------
diff = len(merged) - n_match_rows
print(f"\n* El inner-merge genera {diff} filas extra, "
      "debido a los códigos repetidos en df_xy.")


Filas en df_itur : 9006161
Filas en df_xy   : 697575

=== FILA a FILA (correcto) ===
Coincidencias (filas de df_itur): 697,575
Sin coincidencia                : 8,308,586
Suma                            : 9,006,161

=== CON merge() (duplica filas) ===
Filas en el inner-merge: 697,575

Duplicados en df_xy:
· Filas que se repiten  : 0
· Códigos distintos rep. : 0

* El inner-merge genera 0 filas extra, debido a los códigos repetidos en df_xy.


In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Rutas
path_itur = Path("/mnt/data-r1/data/ITUR/ITUR_resultados_nacional_v1.csv")
path_xy = Path("/mnt/data-r1/JoaquinSalas/Documents/informs/research/2025.06.05ITUR/data/INEGI_CPV2020_n9_XY.csv")
figures_dir = Path("../figures")  # carpeta local relativa
figures_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df_itur = pd.read_csv(path_itur, dtype={"COD_CELDA": str})
df_xy = pd.read_csv(path_xy, dtype={"CODIGO": str})

# Merge
df_xy_renamed = df_xy.rename(columns={"CODIGO": "COD_CELDA"})
merged = pd.merge(df_itur, df_xy_renamed, on="COD_CELDA", how="inner")





In [6]:
# Histograma completo
plt.figure()
plt.hist(df_itur["RESUL_ITUR"].dropna(), bins=50)
plt.title("Histogram of RESULT ITUR – Full df_itur")
plt.xlabel("RESULT ITUR")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.savefig(figures_dir / "hist_result_itur_full.png")

# Histograma de coincidencias
plt.figure()
plt.hist(merged["RESUL_ITUR"].dropna(), bins=50)
plt.title("Histogram of RESULT ITUR – Matched COD_CELDA")
plt.xlabel("RESULT ITUR")
plt.ylabel("Frequency")
plt.grid(True)
plt.axis('equal')  # Ensure same scale for both axes
plt.tight_layout()
plt.savefig(figures_dir / "hist_result_itur_matched.png")

plt.close('all')

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np
from pathlib import Path

# Paths
path_itur_dataset = Path("/mnt/data-r1/data/ITUR_dataset.csv")
figures_dir = Path("../figures")
figures_dir.mkdir(parents=True, exist_ok=True)

# Load data
df = pd.read_csv(path_itur_dataset)

# Extract coordinates
x = df["X"].values
y = df["Y"].values

# Compute the point density
xy = np.vstack([x, y])
density = gaussian_kde(xy)(xy)

# Plot with density and 'hot' colormap
plt.figure(figsize=(10, 10))
plt.scatter(x, y, c=density, s=0.1, cmap='hot', edgecolors='none')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("ITUR values location (density)")
plt.grid(True)
plt.axis('equal')
plt.tight_layout()
plt.savefig(figures_dir / "itur_location_density_hot.png", dpi=300)
plt.close()
